In [13]:
import psycopg2
import sys
import pprint
from datetime import date, timedelta
import pandas as pd

import matplotlib.pyplot as plt
import os,sys
import matplotlib.patches as mpatches
from sklearn import preprocessing

#sys.path.append("../Mfig/")
#import mplp
from itertools import chain
import numpy as np
import csv
from collections import defaultdict
from datetime import date, timedelta
from datetime import datetime
from scipy.stats import entropy
%matplotlib inline

## DB connection

In [14]:
def conectionDB():
    conn_string = "dbname='stic' port='5432' user='stic' password='stic2019' host='gpmaster.as-dell.copernic.local'"
    #print ("Connecting to database ->{}".format(conn_string))
    conn = psycopg2.connect(conn_string)
    return conn

# build dictionaries


In [15]:
dict_dpto = dict()
dict_province = dict()
dict_district = dict()
with open('../data/ubigeo_inei.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        #print (row)
        if (str(row[0])!=''):
            dict_dpto[str(row[3]).upper()]=(str(row[0]))
        if (str(row[1])!=''):
            dict_province[(row[4]).upper()]=(row[0]+''+row[1])
        if (str(row[2])!=''):
            dict_district[(row[5]).upper()]=(row[0]+''+row[1]+''+row[2])
        
dict_gender = {'M':1,'F':0,'':-1}
dict_card = {'TD':1,'TC':0,'':-1}
dict_region = {'NORTE':0,'ORIENTE':1,'SUR':2,'CENTRO':3,'SIERRA CENTRAL':4,
               'LIMA CENTRO':5,'LIMA ESTE':6,'LIMA SUR':7,'LIMA NORTE':8,
               'LIMA MODERNA':9,'LIMA PROVINCIA':10,'CALLAO':11,'':12}


In [16]:
columnNames=[ 
        'social_class' ,     #-- 0
        'country_code',      #-- 1
        'amount_usd',        #-- 2 *
        'client_age',        #-- 3 *
        'client_gender',     #-- 4
        'debit_type',        #-- 5 Credit or debit card
        'agency_region',     #-- 9
        'merchant_departement', #-- 10
        ]


## extraction loop

In [17]:
def extract_all (index):
    PLIMIT = 1000000
    POFFSET = 1000000
    #NUM = 93 # Millions of records (92 470 071)
    NUM = 2 #for test 
    i=index
    while (i < NUM):
        print ("Processing {}/{} ...".format(i,NUM))
        query ="""
            SELECT 
                social_class,      -- 0
                amount_usd,        -- 1 *
                client_age,        -- 2 *
                client_gender,     -- 3
                debit_type,        -- 4 Credit or debit card
                agency_region,     -- 5
                merchant_departement, -- 6
                coicop              --7
            FROM 
                public.bbva 
            INNER JOIN 
                client
            ON 
                client.client_id = bbva.client_id
            INNER JOIN 
                mcc
            ON 
                mcc.id = bbva.mcc
            WHERE
                bbva.merchant_id != '00000000'
            ORDER BY 
                bbva.date ASC
            LIMIT {}
            OFFSET {}
        ;
        """.format(PLIMIT,POFFSET*i)
        conn = conectionDB()
        cursor = conn.cursor();
        df = pd.read_sql(query, conn)
        df.fillna(-1, inplace=True)
        df['small_social_class'] = df.apply(lambda row: 1 if (row.social_class<=3) else 0 , axis=1)
        del df['social_class']
        
        df['client_gender'] = df.apply(lambda row: 1 if (row.client_gender=='M') else 0 , axis=1)
        df['debit_type'] = df.apply(lambda row: 1 if (row.debit_type=="TC") else 0 , axis=1)

        
      
        df=df.replace({ "merchant_departement":dict_dpto,
                        "agency_region": dict_region
                         })
        df.fillna(-1, inplace=True)
        
        del df['amount_usd']
        del df['client_age']
        ###############################        
        #  Normalize numeric columns  #
        ###############################
        min_max_scaler = preprocessing.MinMaxScaler()
        
        # Create x, where x the 'scores' column's values as floats
        x_amount_usd = df[['amount_usd']].values.astype(float)
        x_client_age = df[['client_age']].values.astype(float)
        #del df['social_class']
        # Create an object to transform the data to fit minmax processor
        x_scaled_amount_usd = min_max_scaler.fit_transform(x_amount_usd)
        x_scaled_client_age = min_max_scaler.fit_transform(x_client_age)

        # Run the normalizer on the dataframe
        df_normalized_amount_usd = pd.DataFrame(x_scaled_amount_usd)
        df_normalized_client_age = pd.DataFrame(x_scaled_client_age)
        #df['small_social_class'] = df.apply(lambda row: 1 if (df.social_class<=3) else 0 , axis=1)
        
        #######################################        
        #  One hot enconding numeric columns  #
        #######################################
        
        newdf = pd.concat([df_normalized_amount_usd, 
                   df_normalized_client_age,
                   df
                  ], axis=1)
        
        
        
        #newdf['social_class'] = df['small_social_class']
        #del newdf['small_social_class']
        print (newdf.head)
        print (newdf.columns)

        newdf.to_csv (r'../data_output/export_dataframe_{}v6.csv'.format(i), index = None, header=False)
        i+=1

In [18]:
extract_all(0)

Processing 0/2 ...
<bound method NDFrame.head of                0         0  client_gender  debit_type  agency_region  \
0       0.002222  0.345794              0           1              9   
1       0.000241  0.364486              1           1              5   
2       0.000454  0.317757              0           1              9   
3       0.000244  0.327103              0           1              8   
4       0.001783  0.242991              0           1             11   
...          ...       ...            ...         ...            ...   
999995  0.000417  0.299065              1           0              9   
999996  0.000201  0.280374              1           0              5   
999997  0.000453  0.448598              0           0              9   
999998  0.000363  0.420561              0           0              5   
999999  0.002273  0.448598              0           0              9   

       merchant_departement  coicop  small_social_class  
0                        15 